In [23]:
# !pip install pathlib
# !pip install category_encoders

In [5]:
# Required packages
import pathlib

import category_encoders
import pandas as pd
import sklearn.impute
import sklearn.linear_model
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
import yaml

## Loading and Preprocessing Data

Refer to the [sample Jupyter notebook by Luke Merrick](https://gist.github.com/lukemerrick/af14f5b498ddf3900ba77c7bd840fc8c) to learn best ways to efficiently access the data.  The data has been provied in p2p_loans_470k_data/ in this repo.

Luke Merrick has provided steps to load the data using the schema files provided in p2p_loans_470k_data/. This would ensure the data are loaded using the appropraite data type. He provides some guidance on preparing the data, and building an ML model with it.  His data preparation is very simple so you would need to do a more comprehensive data preparation to get hopefully better model prediction results

In [9]:
# define paths to all the files
data_dir = pathlib.Path('p2p_loans_470k_data/')

feature_schema_yaml = data_dir / 'feature_schema.yaml'
label_schema_yaml = data_dir / 'label_schema.yaml'

train_feature_csv = data_dir / 'train' / 'train_features.csv.gz'
train_label_csv = data_dir / 'train' / 'train_labels.csv.gz'
test_feature_csv = data_dir / 'test' / 'test_features.csv.gz'
test_label_csv = data_dir / 'test' / 'test_labels.csv.gz'

In [11]:
# load data into pandas dataset

with feature_schema_yaml.open() as yaml_file:
    feature_schema = yaml.load(yaml_file, Loader=yaml.FullLoader)
with label_schema_yaml.open() as yaml_file:
    label_schema = yaml.load(yaml_file, Loader=yaml.FullLoader)

# use them to intelligently import the data
train_features = pd.read_csv(train_feature_csv, **feature_schema)
train_labels = pd.read_csv(train_label_csv, **label_schema)
test_features = pd.read_csv(test_feature_csv, **feature_schema)
test_labels = pd.read_csv(test_label_csv, **label_schema)

In [15]:
train_labels.loan_status.unique()

['Charged Off', 'Fully Paid']
Categories (2, object): ['Charged Off', 'Fully Paid']

## Data Dictionary 

In [ ]:
Refer to the following to understand what each feature stan

In [14]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351088 entries, 36805548 to 36271262
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   loan_status              351088 non-null  category      
 1   issue_d                  351088 non-null  datetime64[ns]
 2   zip_code_prefix          351088 non-null  object        
 3   grade                    351088 non-null  category      
 4   sub_grade                351088 non-null  category      
 5   installment              351088 non-null  float64       
 6   int_rate                 351088 non-null  float64       
 7   collection_recovery_fee  351088 non-null  float64       
 8   recoveries               351088 non-null  float64       
 9   debt_settlement_flag     351088 non-null  category      
 10  settlement_amount        4098 non-null    float64       
 11  settlement_date          4098 non-null    datetime64[ns]
 12  settlem

In [16]:
train_labels.head()

,loan_status,issue_d,zip_code_prefix,grade,sub_grade,installment,int_rate,collection_recovery_fee,recoveries,debt_settlement_flag,settlement_amount,settlement_date,settlement_percentage,settlement_status,settlement_term,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp
id,,,,,,,,,,,,,,,,,,,,
36805548,Charged Off,2014-12-01,937,A,A3,321.08,6.99,93.8286,521.27,N,NaN,NaT,NaN,NaN,NaN,6611.69,6611.69,872.67,0.0,5217.75
37662224,Charged Off,2014-12-01,850,C,C3,260.20,13.66,222.8382,1237.99,N,NaN,NaT,NaN,NaN,NaN,2281.98,2281.98,339.61,0.0,704.38
37822187,Fully Paid,2014-12-01,077,C,C3,326.53,13.66,0.0000,0.00,N,NaN,NaT,NaN,NaN,NaN,9973.43,9973.43,373.43,0.0,9600.00
37701596,Charged Off,2014-12-01,483,B,B5,332.10,11.99,0.0000,0.00,N,NaN,NaT,NaN,NaN,NaN,6957.45,6957.45,1562.16,0.0,5395.29
37800722,Charged Off,2014-12-01,331,D,D5,468.17,17.86,272.7990,1515.55,N,NaN,NaT,NaN,NaN,NaN,5746.89,5746.89,1603.20,0.0,2628.14


In [17]:
train_labels.describe()

,installment,int_rate,collection_recovery_fee,recoveries,settlement_amount,settlement_percentage,settlement_term,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp
count,351088.000000,351088.000000,351088.000000,351088.000000,4098.000000,4098.000000,4098.000000,351088.000000,351088.000000,351088.000000,3.510880e+05,351088.000000
mean,420.115238,12.629083,18.402367,123.076300,2998.827230,46.968614,6.140068,13758.528821,13751.892402,2000.899161,9.619749e-01,11633.591377
std,256.784206,3.900596,100.572493,598.764747,2491.401206,10.910215,7.741453,8978.211423,8974.182438,1630.008271,7.220580e+00,7772.177487
min,14.010000,5.930000,0.000000,0.000000,44.210000,15.370000,0.000000,0.000000,0.000000,0.000000,-1.800000e-09,0.000000
25%,232.470000,9.670000,0.000000,0.000000,1262.237500,44.352500,0.000000,7102.820001,7099.920000,903.490000,0.000000e+00,6000.000000
50%,350.740000,12.490000,0.000000,0.000000,2342.715000,45.010000,1.000000,11634.704999,11624.600000,1588.505000,0.000000e+00,10000.000000
75%,549.940000,15.100000,0.000000,0.000000,3900.000000,50.000000,12.000000,18192.110265,18181.290000,2583.045000,0.000000e+00,15300.000000
max,1409.990000,26.060000,6124.938000,34294.100000,22896.570000,521.350000,36.000000,51067.368008,51067.370000,16602.700000,3.676000e+02,35000.000000


In [13]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351088 entries, 36805548 to 36271262
Data columns (total 61 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   loan_amnt                       351088 non-null  float64       
 1   emp_title                       327686 non-null  object        
 2   emp_length                      331440 non-null  category      
 3   home_ownership                  351088 non-null  category      
 4   annual_inc                      351088 non-null  float64       
 5   desc                            68438 non-null   object        
 6   purpose                         351088 non-null  category      
 7   title                           351083 non-null  object        
 8   addr_state                      351088 non-null  category      
 9   dti                             351088 non-null  float64       
 10  delinq_2yrs                     351088 non-null

In [18]:
train_features.head()

,loan_amnt,emp_title,emp_length,home_ownership,annual_inc,desc,purpose,title,addr_state,dti,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,fico_range_midpoint
id,,,,,,,,,,,,,,,,,,,,,
36805548,10400.0,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,NaN,credit_card,Credit card refinancing,CA,14.92,...,0.0,4.0,83.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,712.0
37662224,7650.0,Technical Specialist,< 1 year,RENT,50000.0,NaN,debt_consolidation,Debt consolidation,AZ,34.81,...,0.0,2.0,100.0,0.0,0.0,82331.0,64426.0,4900.0,64031.0,687.0
37822187,9600.0,Admin Specialist,10+ years,RENT,69000.0,NaN,debt_consolidation,Debt consolidation,NJ,25.81,...,0.0,3.0,100.0,0.0,0.0,52490.0,38566.0,21100.0,24890.0,682.0
37701596,10000.0,Investment Consultant,8 years,RENT,90000.0,NaN,debt_consolidation,Debt consolidation,MI,8.44,...,0.0,0.0,100.0,0.0,0.0,24200.0,23723.0,21200.0,0.0,677.0
37800722,12975.0,Sales,10+ years,RENT,60000.0,NaN,house,Home buying,FL,22.42,...,0.0,4.0,89.5,0.0,0.0,42943.0,17281.0,5500.0,27243.0,682.0


## Target feature

### A regression model that predicts if a customer would default or not
The ***loan_status*** feature in the ***train_label dataset*** is the <u>TARGET FEATURE </u>.  A loan is either assigned the  ***'Charged Off'*** label or  ***'Fully Paid'***. 

Loans labeled <strong><span style='color:red;'>'Charged Off'</span></strong> are loans LendingClub have deemed unlikely to be repaid because the borrower has been deliquent.

Loans labeled <strong><span style='color:green;'>'Fully Paid'</span></strong> are loans that were fully paid.

In [19]:
train_labels.loan_status.unique()

['Charged Off', 'Fully Paid']
Categories (2, object): ['Charged Off', 'Fully Paid']

### A customer risk categorization model using the FICO 5 levels rating (Poor, Fair, Good, Very Good, Exceptional) 
The ***fico_range_midpoint*** feature in the ***train_feature dataset*** is the <u>TARGET FEATURE </u>. Note this variable is currently numeric and would need to be converted into a five level categorical variable.   

Details on the FICO score can be found [here](https://www.myfico.com/credit-education/credit-scores).

**NB:** when addressing the task you should also exclude the loan_status feature from the train

In [24]:
train_features.fico_range_midpoint.describe()

count    351088.000000
mean        695.608336
std          29.920541
min         662.000000
25%         672.000000
50%         687.000000
75%         712.000000
max         847.500000
Name: fico_range_midpoint, dtype: float64

### A customer risk categorization model using the FICO 5 levels rating (Poor, Fair, Good, Very Good, Exceptional) 

**Image Source:** [https://www.myfico.com/credit-education/credit-scores](https://www.myfico.com/credit-education/credit-scores)

![title](assets/FICO_credit_score_grouping.PNG)

## Resource materials and references

1. [Sample Jupyter Notebook containing simple ML model](https://gist.github.com/lukemerrick/af14f5b498ddf3900ba77c7bd840fc8c)
2. [Detailed data dictionary for p2p loan data](https://www.kaggle.com/jonchan2003/lending-club-data-dictionary)
3. [Data dictionary in Excel format](https://resources.lendingclub.com/LCDataDictionary.xlsx)
4. [Data source](https://github.com/fiddler-labs/p2p-lending-data)